In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json

In [44]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [45]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [46]:
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

search_button.click()

In [47]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword
keyword = "@travellertimmy"
searchbox.send_keys(keyword)

In [48]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Remove the "@" symbol
    keyword = keyword[1:]
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')
first_result.click()

In [49]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(3)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration

In [50]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all image elements that match the specific class in the current soup
    elements = soup.find_all('a', class_='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')

    # Extract the href attributes and filter URLs that start with "/p/" or "/p/CZK8AFkFGRa/"
    post_urls.extend([element['href'] for element in elements if element['href'].startswith('/p/')])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

unique_post_urls

['/p/CIo3E4JjuRn/',
 '/p/CIpEFVFDNI1/',
 '/p/CIs8-dKh5rI/',
 '/p/CIo2s9JjuOh/',
 '/p/CIpGvp8DW5H/',
 '/p/CIrREw8h8pH/',
 '/p/CIpIgpxj74D/',
 '/p/CIo1-b3D3MV/',
 '/p/CIpFlILDSm8/']

In [51]:
# Define the query parameters to add
query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:

    # Get the current URL of the page
    current_url = driver.current_url

    # Append the query parameters to the current URL
    modified_url = "https://www.instagram.com/" + url + "?" + query_parameters
    
    #Get URL
    
    driver.get(modified_url)
    
    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(3)
    
    # Find the <pre> tag containing the JSON data
    pre_tag = driver.find_element_by_xpath('//pre')

    # Extract the JSON data from the <pre> tag
    json_script = pre_tag.text

    # Parse the JSON data
    json_parsed = json.loads(json_script)
    



C:\Users\Victor\AppData\Local\Temp\ipykernel_6848\218181876.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pre_tag = driver.find_element_by_xpath('//pre')


In [53]:
json_parsed

{'items': [{'taken_at': 1607656455,
   'pk': 2461523212963228092,
   'id': '2461523212963228092_44615660731',
   'device_timestamp': 1607656258272883,
   'client_cache_key': 'MjQ2MTUyMzIxMjk2MzIyODA5Mg==.2',
   'filter_type': 0,
   'caption_is_edited': True,
   'like_and_view_counts_disabled': False,
   'strong_id__': '2461523212963228092_44615660731',
   'is_reshare_of_text_post_app_media_in_ig': False,
   'is_post_live_clips_media': False,
   'deleted_reason': 0,
   'integrity_review_decision': 'pending',
   'has_shared_to_fb': 0,
   'is_unified_video': False,
   'should_request_ads': False,
   'is_visual_reply_commenter_notice_enabled': True,
   'commerciality_status': 'not_commercial',
   'explore_hide_comments': False,
   'shop_routing_user_id': None,
   'can_see_insights_as_brand': False,
   'is_organic_product_tagging_eligible': False,
   'has_liked': False,
   'has_privately_liked': False,
   'like_count': 14,
   'facepile_top_likers': [{'pk': 308742621,
     'pk_id': '30874262